# 基站覆盖问题

本笔记本旨在使用 OptVerse 来复现 [Gurobi 基站覆盖问题](https://github.com/Gurobi/modeling-examples/tree/master/cell_tower_coverage/cell_tower.ipynb)。
此示例演示了如何解决基站选址问题，以为尽可能多的人群提供信号覆盖。


包含本示例和其他示例的代码库可通过 [CodeHub](link) 获取

## 问题描述

我们首先回顾[Gurobi 基站覆盖问题](https://github.com/Gurobi/modeling-examples/tree/master/cell_tower_coverage/cell_tower.ipynb)描述。

一家电信公司需要建设一组基站来为某个城市的居民提供信号覆盖。已经确定了一些可以建设基站的潜在地点。基站具有固定的覆盖范围，由于预算限制，只能建设有限数量的基站。在这些限制条件下，公司希望为尽可能多的人口提供覆盖。为了简化问题，公司将要覆盖的区域划分为一组区域，每个区域都有已知的人口数量。目标是选择在哪些潜在地点建设基站，以覆盖尽可能多的人口。

基站覆盖问题是最大覆盖选址问题的一个特例。它也与集合覆盖问题相关。

## 求解方法

以下部分取自 [Gurobi 基站覆盖示例](https://github.com/Gurobi/modeling-examples/tree/master/cell_tower_coverage/cell_tower.ipynb)。

数学规划方案通常把复杂的应用问题通过数学语言，将其描述为数学优化模型，然后调用运筹优化求解器进行求解。通常，一个数学优化模型由以下5个因素组成：

* 集合和索引
* 参数
* 决策变量
* 目标函数
* 约束条件

我们接下来为上述基站覆盖问题建立一个混合整数线性规划模型。

## 数学模型

### 集合及下标集

$i \in T$：建设基站的潜在地点的索引和集合。

$j \in R$：区域的索引和集合。

$G(T,R,E)$：定义在潜在基站建设地点集合 $T$、要覆盖的区域集合 $R$ 和边集合 $E$ 上的二部图，其中如果区域 $j \in R$ 可以被地点 $i \in T$ 的基站覆盖，则存在边 $(i,j) \in E$。

### 参数

$c_{i} \in \mathbb{R}^+$：在地点 $i$ 建设基站的成本。

$p_{j} \in \mathbb{N}$：区域 $j$ 的人口数量。

### 决策变量

$covered_{j} \in \{0, 1 \}$：如果区域 $j$ 被覆盖，则此变量等于 1；否则为 0。

$build_{i} \in \{0, 1 \}$：如果建设基站 $i$，则此变量等于 1；否则为 0。

### 目标函数

- **人口覆盖**：我们寻求最大化基站覆盖的总人口。

$$\text{Max} \quad Z = \sum_{j \in R} p_{j} \cdot covered_{j}$$

### 约束条件

- **覆盖约束**：对于每个区域 $j \in R$，确保必须选择至少一个覆盖该区域的基站。

$$\sum_{(i,j) \in E} build_{i} \geq covered_{j} \quad \forall j \in R$$

- **预算约束**：我们需要确保建设基站的总成本不超过分配的预算。

$$\sum_{i \in T} c_{i} \cdot build_{i} \leq \text{budget}$$

## Python 实现

我们接下来展示如何通过Python + OptVerse 解决上述优化问题。

### 环境配置

在使用 OptVerse Python 环境之前，用户需要导入Python包。

In [ ]:
from optvpy import *

### 数据准备

我们首先准备构建模型所需的数据。
为了方便展示，我们将明确构造出这些数据。
此示例考虑 6 个基站和 9 个区域的二部图。

In [ ]:
# 参数
budget = 20  # 预算（百万美元）
regions = [0, 1, 2, 3, 4, 5, 6, 7, 8]  # 区域列表
population = [523, 690, 420, 1010, 1200, 850, 400, 1008, 950]  # 各区域人口

sites = [0, 1, 2, 3, 4, 5]  # 基站地点列表
# 每个基站的覆盖区域
coverage = {
    0: [0, 1, 5],
    1: [0, 7, 8],
    2: [2, 3, 4, 6],
    3: [2, 5, 6],
    4: [0, 2, 6, 7, 8],
    5: [3, 4, 8]
}
cost = [4.2, 6.1, 5.2, 5.5, 4.8, 9.2]  # 各基站建设成本（百万美元）

num_sites = len(sites)      # 基站数量
num_regions = len(regions)  # 区域数量

# 为约束创建覆盖矩阵
coverage_matrix = []
for r in regions:
    covering_sites = []  # 覆盖区域 r 的基站列表
    for s in sites:
        if r in coverage[s]:
            covering_sites.append(s)
    coverage_matrix.append(covering_sites)

### 建立模型

在模型创建之前，我们需要先创建一个OPTVEnv对象，它需要在模型创建时传给所建的模型。

In [ ]:
env = OPTVEnv()          # 创建优化环境
model = OPTVModel(env)   # 创建优化模型

### 添加变量

接下来，我们新建 `build`和 `is_covered` 两组决策变量。
我们通过变量添加接口来设置它们在目标函数中的系数。

In [ ]:
# 添加建设变量
build_lb = num_sites * [0]            # 下界
build_ub = num_sites * [1]            # 上界
build_obj = num_sites * [0]           # 目标函数系数（最大化问题中不直接使用）
build_type = num_sites * [OPTV_BINARY]  # 变量类型：二进制
build_name = [f"build[{i}]" for i in range(num_sites)]  # 变量名称

build = model.AddVars(build_lb, build_ub, build_obj, build_type, build_name)

# 添加覆盖变量
covered_lb = num_regions * [0]            # 下界
covered_ub = num_regions * [1]            # 上界
covered_obj = num_regions * [0]           # 目标函数系数（稍后单独设置）
covered_type = num_regions * [OPTV_BINARY]  # 变量类型：二进制
covered_name = [f"covered[{i}]" for i in range(num_regions)]  # 变量名称

is_covered = model.AddVars(covered_lb, covered_ub, covered_obj, covered_type, covered_name)

### 添加约束

现在我们向模型添加约束。

In [ ]:
# 覆盖约束：覆盖区域 r 的基站总和 >= is_covered[r]
coverage_expr = [sum([build[s] for s in coverage_matrix[r]]) - is_covered[r] for r in range(num_regions)]
coverage_lb = num_regions * [0]        # 下界
coverage_ub = num_regions * [OPTV_INF] # 上界
coverage_name = [f"coverage[{i}]" for i in range(num_regions)]  # 约束名称

model.AddConstrs(coverage_expr, coverage_lb, coverage_ub, coverage_name)

# 预算约束：成本总和 <= 预算
budget_expr = [sum([cost[s] * build[s] for s in range(num_sites)])]
budget_lb = [-OPTV_INF]  # 下界
budget_ub = [budget]     # 上界
budget_name = ["budget"] # 约束名称

model.AddConstrs(budget_expr, budget_lb, budget_ub, budget_name)

### 设置目标函数

设置目标函数以最大化人口覆盖。

In [ ]:
# 创建目标函数表达式：最大化 人口 * 覆盖状态 的总和
obj_expr = sum([population[i] * is_covered[i] for i in range(num_regions)])
model.SetObjective(obj_expr, sense=-1) # -1 表示最大化

### 优化模型

最后，我们可以求解所建的模型了

In [ ]:
model.Optimize() 

## 优化结果分析

本优化方案的结果显示了在 2000 万美元预算下可以覆盖的最大人口数量，并确定在哪些地点建设基站。
方案详情如下：

### 基站建设方案

In [ ]:
# 显示基站建设计划
for tower in build:
    if (abs(tower.Get(OPTVDblAttr.X)) > 1e-6):
        print(f"\n 在地点 {tower.Index()} 建设基站。")

### 覆盖方案

本方案也确定了哪些区域被所建基站覆盖：

In [ ]:
# 显示区域覆盖情况
for i in range(num_regions):
    if (abs(is_covered[i].Get(OPTVDblAttr.X)) > 1e-6):
        print(f"\n 区域 {i}（人口 {population[i]}）被覆盖。")

### 性能指标

计算覆盖率和预算利用率指标。

In [ ]:
# 计算总覆盖人口
total_population = sum(population)  # 总人口
total_covered = 0                   # 被覆盖的总人口

for i in range(num_regions):
    if (abs(is_covered[i].Get(OPTVDblAttr.X)) > 1e-6):
        total_covered += population[i]

coverage_percentage = round(100 * total_covered / total_population, 2)
print(f"\n 基站建设计划的人口覆盖率为：{coverage_percentage}%")

# 计算预算消耗
total_cost = 0  # 总建设成本
for tower in build:
    if (abs(tower.Get(OPTVDblAttr.X)) > 1e-6):
        total_cost += cost[tower.Index()]

budget_consumption = round(100 * total_cost / budget, 2)
print(f"\n 基站建设计划的预算消耗百分比为：{budget_consumption}%")

## 结论

在此示例中，我们解决了基站覆盖问题，目标是在满足预算约束的情况下建设基站，为尽可能多的人口提供信号覆盖。
我们学习了如何将问题制定为 MIP 模型。
此外，我们还学习了如何使用 OptVerse Python API 实现 MIP 模型并求解。